# Title

Intro / research explenation <br>
Link to datasets

In [421]:
import yaml
import re
import pandas as pd

## 1. Load files and create data frames

In [422]:
with open("config.yaml", "r") as stream:
    config_values = yaml.safe_load(stream)
    temperature_path = config_values['temperature_data_path']
    townships_path = config_values['townships_data_path']
    mortality_path = config_values['mortality_data_path']
    weather_stations_coordinates = config_values['weather_stations_coordinates_path']
    
# Create data frames
temperature_df = pd.read_csv(temperature_path, skiprows=11)
townships_df = pd.read_excel(townships_path)
mortality_df = pd.read_csv(mortality_path, sep=';')
ws_coordinates_df = pd.read_excel(weather_stations_coordinates)

## 2. Inspect data
This sections handles the data inspection of the data frames.

### Temperature

In [423]:
temperature_df.head()

,STN,YYYY,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEAR
0,235,1906,40,29,38,68,110,129,162,170,151,123,87,21,94
1,235,1907,26,8,45,73,116,132,137,150,140,115,69,42,88
2,235,1908,0,36,31,55,114,145,160,152,136,101,51,19,83
3,235,1909,17,6,16,71,101,120,139,150,129,112,60,33,80
4,235,1910,39,39,49,72,112,153,144,159,136,103,44,55,92


In [424]:
temperature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STN     944 non-null    int64 
 1   YYYY    944 non-null    int64 
 2      JAN  944 non-null    object
 3      FEB  944 non-null    object
 4      MAR  944 non-null    object
 5      APR  944 non-null    object
 6      MAY  944 non-null    object
 7      JUN  944 non-null    object
 8      JUL  944 non-null    object
 9      AUG  944 non-null    int64 
 10     SEP  944 non-null    object
 11     OCT  944 non-null    object
 12     NOV  944 non-null    object
 13     DEC  944 non-null    object
 14    YEAR  944 non-null    object
dtypes: int64(3), object(12)
memory usage: 110.8+ KB


The temperature data frame has a total of 944 entries. There are 15 columns. 12 of these columns are month columns (Jan - Dec). These column names contain leading spaces which should be removed. The remaining three columns are the station number, the year, and the year average. To get the real temperatures, the values need to be divided by 10. What stands out is that, regarding the temperature columns, only August has the correct data type. Also, the column names of the station number and the year could be a bit more descriptive, whereas the name of the year average column (which we don't need) is a little misleading. These issues will be fixed during the data wrangling.

In [425]:
temperature_df.isna().sum()

STN       0
YYYY      0
   JAN    0
   FEB    0
   MAR    0
   APR    0
   MAY    0
   JUN    0
   JUL    0
   AUG    0
   SEP    0
   OCT    0
   NOV    0
   DEC    0
  YEAR    0
dtype: int64

The temperature data frame is not missing any data.

### Townships

In [426]:
townships_df.head()

,GM_CODE,GM_NAAM,geometry
0,GM0003,Appingedam,<Polygon><outerBoundaryIs><LinearRing><coordin...
1,GM0005,Bedum,<Polygon><outerBoundaryIs><LinearRing><coordin...
2,GM0007,Bellingwedde,<Polygon><outerBoundaryIs><LinearRing><coordin...
3,GM0009,Ten Boer,<Polygon><outerBoundaryIs><LinearRing><coordin...
4,GM0010,Delfzijl,<Polygon><outerBoundaryIs><LinearRing><coordin...


In [427]:
townships_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   GM_CODE   431 non-null    object
 1   GM_NAAM   431 non-null    object
 2   geometry  431 non-null    object
dtypes: object(3)
memory usage: 10.2+ KB


The township data frame exists of 431 entries. There are only 3 columns: the township code (which match the codes in the mortality data), the township name and the geometry. The data types of the columns are good as they are. The geometry column contains data to create a polygon. However, in this research we don't need a complete polygon, but we need the center point (lat/lon) of the polygons. Furthermore, the column names could be a bit more descriptive. These issues will be fixed during the data wrangling part.

In [428]:
townships_df.isna().sum()

GM_CODE     0
GM_NAAM     0
geometry    0
dtype: int64

The township data frame is not missing any data.

### Mortality

In [429]:
mortality_df.head()

,ID,RegioS,Perioden,Overledenen_3
0,0,NL01,2002MM01,13469
1,1,NL01,2002MM02,11735
2,2,NL01,2002MM03,13281
3,3,NL01,2002MM04,11968
4,4,NL01,2002MM05,11623


In [430]:
mortality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167076 entries, 0 to 167075
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             167076 non-null  int64 
 1   RegioS         167076 non-null  object
 2   Perioden       167076 non-null  object
 3   Overledenen_3  129332 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.1+ MB


The mortality data frame exists of 167076 entries. There are 4 columns: an ID, a region code, a period and the number of deceased people. The ID column should be set as actual ID of the data frame. The values in the region code column can start with the following prefixes: NL (whole Netherlands), LD (landsdeel: country part), PV (province) and GM (gemeente: township). For this research we only need the records with a GM-code. The period column contains a combination of a year and a month number. This column should be separated in two columns: year and month. Furthermore, the data type of the deceased column should be changed to the right data type. Lastly, the column names of this data frame could be a bit more descriptive. These issues will be fixed during the data wrangling part.

In [431]:
mortality_df.isna().sum()

ID                   0
RegioS               0
Perioden             0
Overledenen_3    37744
dtype: int64

It appears that there are 37744 entries that are missing the number of deceased people. For this research the choice has been made to leave out the townships with missing data, to get the most reliable results.

### Weather stations coordinates

In [432]:
ws_coordinates_df

,Station,Lat,Lon
0,235,52.927569,4.777465
1,270,53.222570,5.768726
2,280,53.125899,6.584522
3,290,52.273230,6.881168
4,240,52.314998,4.766546
5,260,52.103508,5.176900
6,344,51.924790,4.473661
7,310,51.452907,3.567217
8,370,51.501198,5.474265
9,380,50.905617,5.761880


In [433]:
ws_coordinates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Station  10 non-null     int64  
 1   Lat      10 non-null     float64
 2   Lon      10 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 368.0 bytes


The weather stations coordinates data frame contains the coordinates (lat/lon) of the 10 weather stations that are included in this research. There is no need to clean/modify this data frame.

## 3. Wrangle data
This section handles the wrangling of the data sets, as described in the data inspection part.

### Temperature

In [434]:
# Remove leading spaces in column names
temperature_df.columns = temperature_df.columns.str.strip()

# Drop YEAR column (year average that we don't need)
temperature_df.drop('YEAR', axis=1, inplace=True)

# Rename station and year column
temperature_df.rename(columns={'STN': 'Station', 'YYYY': 'Year'}, inplace=True)

# Select only the years from 2000
temperature_df = temperature_df.loc[temperature_df['Year'] >= 2002]

# Change data type of month columns to int
temperature_df.loc[:, 'JAN':'DEC'] = temperature_df.loc[:, 'JAN':'DEC'].astype('int64')

# Correct the temperatures by dividing them by 10
temperature_df.loc[:, 'JAN':'DEC'] = temperature_df.loc[:, 'JAN':'DEC'].div(10)

### Townships

In [435]:
def calculate_centroid(coordinates):
    '''Function that takes a set of coordinates of a polygon and calculates the center point'''
    only_numbers = [re.sub(r'[^0-9.]', '', coord) for coord in coordinates][0:-1]
    y = [float(only_numbers[i]) for i in range(0, len(only_numbers), 2)]
    x = [float(only_numbers[i]) for i in range(1, len(only_numbers), 2)]
    l = len(x)
    return sum(x) / l, sum(y) / l

In [436]:
# Rename columns
townships_df.rename(columns={'GM_CODE': 'Code', 'GM_NAAM': 'Name'}, inplace=True)

# Calculate center point of the polygon
townships_df['geometry'] = townships_df['geometry'].astype(str).str.extract(r"<coordinates>(.*)</coordinates>")
townships_df['geometry'] = townships_df['geometry'].str.replace('0.0 ', '', regex=True)
townships_df['geometry'] = townships_df['geometry'].str.replace('0.0', '', regex=True)
townships_df['geometry'] = townships_df['geometry'].str.split(',').apply(calculate_centroid)
townships_df[['Latitude', 'Longitude']] = pd.DataFrame(townships_df['geometry'].tolist(), index=townships_df.index)

# Drop geometry column, since it's not needed anymore
townships_df.drop('geometry', axis=1, inplace=True)

### Mortality

In [439]:
# Set ID column as index
mortality_df.set_index("ID", inplace = True)

# Drop all the entries with a period containing 'JJ'. These are year totals and are not used
mortality_df = mortality_df[mortality_df['Perioden'].str.contains('JJ') == False]

# Split period column into year and month column and drop old period column
mortality_df[['Year','Month']] = mortality_df['Perioden'].str.split('MM',expand=True)
mortality_df.drop('Perioden', axis=1, inplace=True)

# Rename columns
mortality_df.rename(columns={'RegioS': 'Code', 'Overledenen_3': 'Deceased'}, inplace=True)

# Filter township records (codes starts with GM)
mortality_df = mortality_df.loc[mortality_df['Code'].str[:2] == 'GM']

# Remove all entries of townships that are missing data
total_townships = mortality_df['Code'].nunique()
townships_missing_data = mortality_df.loc[mortality_df['Deceased'].isna()]['Code'].unique()
mortality_df.drop(mortality_df[mortality_df['Code'].isin(townships_missing_data)].index, inplace=True)

# Correct data type of deceased column
mortality_df[['Deceased', 'Year', 'Month']] = mortality_df[['Deceased', 'Year', 'Month']].apply(pd.to_numeric)

In [385]:
print(f'From the total of {total_townships} townships are {len(townships_missing_data)} missing deceased data. This leaves a total of {total_townships - len(townships_missing_data)} townships that are included in this  research.')

From the total of 555 townships are 267 missing deceased data. This leaves a total of 288 townships that are included in this  research.


## 4. Merge dataframes

In [451]:
# Merge the mortality and townships data frames
mortality_township_df = pd.merge(mortality_df, townships_df, how='inner', on='Code')

mortality_township_df.head()

,Code,Deceased,Year,Month,Name,Latitude,Longitude
0,GM1680,21,2002,1,Aa en Hunze,53.043303,6.716957
1,GM1680,16,2002,2,Aa en Hunze,53.043303,6.716957
2,GM1680,23,2002,3,Aa en Hunze,53.043303,6.716957
3,GM1680,13,2002,4,Aa en Hunze,53.043303,6.716957
4,GM1680,20,2002,5,Aa en Hunze,53.043303,6.716957


## Analyse data (create graphs, etc.)

## Statistics